# Stroke prediction

## Introduction

The goal of this model is to predict if a person is going to suffer a stroke.

Since false negative are more important (false positives can be checked at the hospital), we are going to add more weight on those in the loss function. 

### Background

A stroke occurs when the blood supply to part of your brain is interrupted or reduced, preventing brain tissue from getting oxygen and nutrients. Brain cells begin to die in minutes. A stroke is a medical emergency, and prompt treatment is crucial. Early action can reduce brain damage and other complications. The good news is that many fewer Americans die of stroke now than in the past. Effective treatments can also help prevent disability from stroke.

- [ ] Try using SMOTETOMEK etc.
- [ ] Try undersampling methods.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn
import imblearn

from bokeh.io import push_notebook, output_notebook, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
output_notebook()

sns.set_style('whitegrid')
cmap = sns.cm.mako_r

%matplotlib inline

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.utils import check_random_state
np.random.seed(42)
assert check_random_state is not None, "Set numpy seed for determinism."

In [ ]:
df = pd.read_csv('../dataset/stroke.csv').drop(columns=['id'])

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
categorical_cols = df.select_dtypes(include=object).columns
binary_cols = ['stroke', 'hypertension', 'heart_disease']
numerical_cols = df.select_dtypes(include=np.number).columns.drop(binary_cols)
print(f'Numerical variables: {len(numerical_cols)}')
print(f'Categorical variables: {len(categorical_cols) + len(binary_cols)}')

In [ ]:
df[categorical_cols].describe()

## Preprocessing

Age should be rounded to a natural.

In [ ]:
df['age'] = df['age'].apply(round)

Binary categorical variables should be modeled properly.

In [ ]:
def binary2Categorical(df, cols):
  for c in cols:
    df[c] = pd.cut(df[c], bins=[-0.5,0.5,1.5], labels=['no', 'yes'])

In [ ]:
binary_cols = ['hypertension', 'heart_disease']    
binary2Categorical(df, binary_cols)
categorical_cols = df.select_dtypes(include=[object, 'category']).columns

### Dealing with outliers

In [ ]:
df[numerical_cols].hist(bins=20, layout=(2,3), figsize=(9,6))

In [ ]:
fig, axes= plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 4]}, figsize=(9,4))
df.boxplot(column='age',ax=axes[0])
df.hist(bins=25, column='age', ax=axes[1], legend=True)

Here we could have used IQR filtering but it would have removed the values of the extrem of the interval which is no good.

After some research (see [Overweight & Obesity](https://www.cdc.gov/obesity/adult/defining.html)), we consider a BMI less than 15 and greater than 60 an outlier.

In [ ]:
df['bmi'] = df['bmi'].apply(lambda bmi_value: bmi_value if 15 < bmi_value < 60 else np.nan)

There is only one person with 'Other' gender so we will consider it an outlier.

In [ ]:
df = df[df['gender'] != 'Other']

### Replace Low Frequency Categorical Variables

In [ ]:
#def replaceLowFreq(df, threshold=0.1, replacement='other'):
#    f = lambda x: x.map(x.value_counts(normalize=True)) >= threshold
#    r = df[categorical_cols].where(df[categorical_cols].apply(f), replacement)
#    df[categorical_cols] = r
#
#replaceLowFreq(df, threshold=0.1)

### Imputation

In [ ]:
def displayMissingValues():
    plt.title('Amount Of Missing Values',fontsize=14, y=-0.2)
    ax = sns.heatmap(pd.DataFrame(df.isna().sum()),annot=True, fmt='d', vmin=0, vmax=df.shape[0])
displayMissingValues()

A naive imputation such as the *mean of all non missing bmi* will not work since 'bmi' depends on other variables e.g. 'age', 'gender', etc.

```python
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

X = np.array(df.bmi).reshape(-1,1)
df.bmi = imp_mean.fit_transform(X)
```

This is why we are going to impute the missing bmi values using a Decision Tree based with variables age and gender.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

imp_dt = DecisionTreeRegressor(random_state=42)

pipe = \
  Pipeline(steps=[('scale',StandardScaler()), 
                  ('imputer', imp_dt)])

X = df[['age','gender','bmi']].copy()
X.gender = X.gender.replace({'Male':0,'Female':1,'Other':-1}).astype(np.uint8)

Missing = X[X.bmi.isna()]
X = X[~X.bmi.isna()]
Y = X.pop('bmi')
pipe.fit(X,Y)
predicted_bmi = pd.Series(pipe.predict(Missing[['age','gender']]), index=Missing.index)
df.loc[Missing.index,'bmi'] = predicted_bmi

In [ ]:
displayMissingValues()

## Visualization

First, let's display the data grouped by target class.

In [ ]:
fig1, ax1 = plt.subplots()
ax1.pie(df['stroke'].value_counts(), explode=[0, 0.3], labels=['no stroke', 'stroke'], autopct='%1.1f%%',
        shadow=True, startangle=180)
ax1.set_title('Stroke class proportion', loc='center', y=1.0, fontsize=14, fontweight='bold')
ax1.axis('equal')

plt.show()

The dataset is **imbalanced**. We will take this into account in the next sections by using a resampling method such as SMOTE.

In [ ]:
from math import pi

len_data = len(df)
len_m = len(df[df["gender"]=="Male"])
len_w = len_data - len_m

men_stroke = len(df.loc[(df["stroke"]==1)&(df['gender']=="Male")])
men_no_stroke = len_m - men_stroke

women_stroke = len(df.loc[(df["stroke"]==1) & (df['gender']=="Female")])
women_no_stroke = len_w - women_stroke

x = {
    'Men with stroke': men_stroke,
    'Women with stroke': women_stroke,
    'Men healthy': men_no_stroke,
    'Women healthy': women_no_stroke
}
          
data = pd.Series(x).reset_index(name='value').rename(columns={'index':'label'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, toolbar_location=None,
           tools="hover", tooltips="@label: @value", x_range=(-0.5, 1.0))
p.title.text = 'Stroke class grouped by gender'
p.title.align = 'center'
p.title.text_color = 'black'
p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='label', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)

In [ ]:
df.groupby(['stroke']).sum().plot(kind='pie', subplots=True, figsize=(10,5), startangle=180)

The classes representation seems to be proportioned when partition by each numerical variable.

Next, we are going to have a look at the categorical variables.

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(20,20))

for i, c in enumerate(categorical_cols):
  ax = axes.reshape(-1)[i]
  ct = pd.crosstab(index=df[c], columns=df['stroke'],normalize='index')
  a = ct.plot(kind='bar', stacked=True,ax=ax)

# Do not show empty plots
for ax in axes.reshape(-1)[len(categorical_cols):]:
    ax.set_visible(False)
    
plt.tight_layout()

- Stroke doesn't seem to be highly correlated to smoke since the proportion of person having a strok is farily the same among the different smoking status.
- Men are more prone to stroke than women.
- The gender does not discriminite a person having a stroke or not.
- Hypertension and heart disease does affect a person having a stroke.
- Rural people is less prone to stroke.

Let's have a look at the KDE plots of the numerical variables by stroke.

In [ ]:
fig = plt.figure(figsize=(12, 12), dpi=150)
gs = fig.add_gridspec(4, 3)
gs.update(wspace=0.1, hspace=0.4)

plot = 0
for row in range(0, 1):
    for col in range(0, 3):
        locals()["ax"+str(plot)] = fig.add_subplot(gs[row, col])
        locals()["ax"+str(plot)].tick_params(axis='y', left=False)
        locals()["ax"+str(plot)].get_yaxis().set_visible(False)
        for s in ["top","right","left"]:
            locals()["ax"+str(plot)].spines[s].set_visible(False)
        plot += 1

plot = 0

s = df[df['stroke'] == 1]
ns = df[df['stroke'] == 0]

for feature in numerical_cols:
        sns.kdeplot(s[feature], ax=locals()["ax"+str(plot)], color='#0f4c81', shade=True, linewidth=1.5, ec='black',alpha=0.9, zorder=3, legend=False)
        sns.kdeplot(ns[feature],ax=locals()["ax"+str(plot)], color='#9bb7d4', shade=True, linewidth=1.5, ec='black',alpha=0.9, zorder=3, legend=False)
        locals()["ax"+str(plot)].grid(which='major', axis='x', zorder=0, color='gray', linestyle=':', dashes=(1,5))
        plot += 1

ax0.set_xlabel('Age')
ax1.set_xlabel('Avg. Glucose Levels')
ax2.set_xlabel('BMI')
        
ax0.text(-20, 0.056, 'Numeric Variables by stroke', fontsize=20, fontweight='bold')

plt.show()

Age looks to be a prominent factor.

## Correlation

In [ ]:
fig = px.parallel_categories(df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type',
       'smoking_status', 'stroke']], color='stroke', color_continuous_scale=px.colors.sequential.Inferno)
fig.show()

In [ ]:
# This is terribly slow...
g = sns.pairplot(df, diag_kind="kde", hue='stroke', corner=True, height=2.0)
g.map_lower(sns.kdeplot, levels=4, color=".2")

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_aux = df.apply(encoder.fit_transform)

plt.figure(figsize=(16,8))
sns.heatmap(df_aux.corr(),cmap="Blues", annot=True);

## Modeling

Since the dataset is imbalanced, a useful baseline to beat is the _the null accuracy_. We will use the inverse since we are predicting the positive case 'stroke'.  

Our baseline will be $\approx 5\%$ for recall of positive stroke.

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

We start by spliting data into training and testing.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns != 'stroke']
y = df['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Encoding

There are multiple ways to make the categorical variables numerical:
- LabelEncoder
- OneHotEncoder
- OrdinalEncoder
- ...

In [ ]:
X_train_one = pd.get_dummies(X_train, columns = categorical_cols, drop_first=True)

### Over-sampling

The learning phase and the subsequent prediction of machine learning algorithms can be affected by the problem of imbalanced data set. The balancing issue corresponds to the difference of the number of samples in the different classes.

In order to fix this problem, we are going to use SMOTE (Synthetic Minority Over-sampling Technique) which is an over-sampling method based on linear interpolation of the k-Nearest Neighbors.

In [ ]:
from imblearn.over_sampling import SMOTE

To pick the best model we are going to use cross-validation.

In [ ]:
from sklearn.model_selection import cross_val_score

def runCrossValidation(pipeline, X, y, folds=10, scoring='f1'):
  scores = cross_val_score(pipeline, X, y, cv=folds, scoring=scoring)
  print(f'Mean F1-score: {scores.mean()}')

### Pipeline: Standarization + SMOTE + Classifier

In [ ]:
from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def getPipeline(classifier):
    scale = StandardScaler()
    smote = SMOTE(random_state=42, k_neighbors=5)
    p = make_pipeline(scale, smote, classifier)
    return p

#### SVC

In [ ]:
svc = SVC(random_state=42)
pipeline = getPipeline(svc) 
runCrossValidation(pipeline, X_train_one, y_train)

#### Random Forest

In [ ]:
rfc = RandomForestClassifier(random_state=42)
pipeline = getPipeline(rfc) 

# TODO NaN ??
runCrossValidation(pipeline, X_train, y_train)

# One-hot encoding
runCrossValidation(pipeline, X_train_one, y_train)

#### Logistic Regression

In [ ]:
lr = LogisticRegression(random_state=42)
pipeline = getPipeline(lr) 
runCrossValidation(pipeline, X_train_one, y_train)

#### KNN

In [ ]:
# TODO hyperparameter
knn = KNeighborsClassifier(n_neighbors=3)
pipeline = getPipeline(knn) 
runCrossValidation(pipeline, X_train_one, y_train)

#### AdaBoost

In [ ]:
# TODO hyperparameter
ada=AdaBoostClassifier(n_estimators=100, random_state=42)
pipeline = getPipeline(ada) 
runCrossValidation(pipeline, X_train_one, y_train)

### Factor Analysis

In [ ]:
import prince

def MCA(n_components):
  mca = prince.MCA(
          n_components=n_components,
          n_iter=100,
          copy=True,
          check_input=True,
          engine='auto',
          random_state=42)
  return mca

def printMCA(X, n_components):
  mca = MCA(n_components)
  mca = mca.fit(X)
    
  plt.plot(np.cumsum(mca.explained_inertia_))
  plt.title('Cumulative sum of inertia')
    
  mca.plot_coordinates(
    X=X,
    ax=None,
    figsize=(12, 12),
    show_row_points=True,
    row_points_size=10,
    show_row_labels=False,
    show_column_points=True,
    column_points_size=30,
    show_column_labels=True,
    legend_n_cols=1)

In [ ]:
printMCA(X_train, n_components=17)

The inertia is really low since MCA only uses categorical variables.

Let's try binning the numerical variables into categorical ones in order to use them on the MCA.

In [ ]:
def binNumericalVariables(X):
  r = X.copy()
  r.drop(columns=['age', 'bmi', 'avg_glucose_level'])
  r['age'] = pd.cut(X_train['age'], bins=[0,13,18,45,60,150], labels=['children', 'teens', 'adults', 'seniors', 'elderly'])
  r['bmi'] = pd.cut(X_train['bmi'], bins=[0,19,25,30,300], labels=['underweight', 'normal', 'overweight', 'obesity'])
  r['avg_glucose_level'] = pd.cut(X_train['avg_glucose_level'], bins=[0,90,160,230,500], labels=['low','normal','high','very high']) 
  return r

The glucose level binning is extracted from this [blog](https://agamatrix.com/blog/normal-blood-sugar-level-chart/).

In [ ]:
X_train_mca = binNumericalVariables(X_train)

In [ ]:
X_train_mca.dtypes

All variables are now encoded as binary or categorical

In [ ]:
printMCA(X_train_mca, n_components=17)

### Pipeline: Standarization + MCA + SMOTE + Classifier

In [ ]:
def getMCAPipeline(classifier):
    scale = StandardScaler()
    mca = MCA(n_components=12) # 80% inertia
    smote = SMOTE(random_state=42, k_neighbors=5)
    p = make_pipeline(scale, mca, smote, classifier)
    return p

#### SVC

In [ ]:
# TODO NaN
svc = SVC(random_state=42)
pipeline = getMCAPipeline(svc) 
runCrossValidation(pipeline, X_train_mca, y_train)

#### Random Forest

In [ ]:
# TODO NaN
rfc = RandomForestClassifier(random_state=42)
pipeline = getMCAPipeline(rfc) 
runCrossValidation(pipeline, X_train_mca, y_train)

## Evaluating the model

Select the best model w.r.t the cross-validation score and evaluate it.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,f1_score

# TODO 

# print(confusion_matrix(y_val, yPredict))
# print(f1_score(y_val, yPredict))